In [1]:
import os, sys

fileDir = os.getcwd()
path = os.path.join(fileDir, "..")
sys.path.append(path)

from utilities import *

# Sample Data

In [2]:
data = [(1, "Alice"),
        (2, "Bob"),
        (3, "Charlie")]

columns = ["id", "name"]

# Neo4j connector test

In [3]:
session = create_spark_session("Neo4j Test", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


## Write

In [4]:
df = session.createDataFrame(data, columns)

options = get_default_options(SparkConnector.NEO4J)
options["node.keys"] = "id"
options["labels"] = ":Person"

spark_write(SparkConnector.NEO4J, df, "Overwrite", options=options)


Dataframe saved to NEO4J


## Read

In [5]:
options = get_default_options(SparkConnector.NEO4J)
options["labels"] = ":Person"

df = spark_read(SparkConnector.NEO4J, session, options=options)
df.show()

Dataframe loaded from neo4j
+----+--------+-------+---+
|<id>|<labels>|   name| id|
+----+--------+-------+---+
|   0|[Person]|    Bob|  2|
|   1|[Person]|  Alice|  1|
|   2|[Person]|Charlie|  3|
+----+--------+-------+---+



## Stop spark context and spark session

In [6]:

session.sparkContext.stop()
session.stop()